In [1]:
import pandas as pd
from lib import search_df
from IPython.display import Markdown as md


/Users/serickson-local/src/ucsb-writing-data/.venv/lib/python3.12/site-packages/sentence_transformers/cross_encoder/CrossEncoder.py:11: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange


In [2]:
def search_display(q: str, df: pd.DataFrame, limit: int = 25):
    result = search_df(q, df, limit)
    for i, row in result.iterrows():
        text = f"({str(row['result_distance'])[:6]}) {row['result_text']}"
        display(md(text))

In [3]:
df = pd.read_csv("data/reflections.csv")
search_display("athletics sports", df[df.first_gen_coll_student == 1])

(14.979) The subject "stereotypes" was mostly tackled during my rhetorical class. Each student had to make an argumentative speech about a topic that they liked and we had to "pretend" that we were a specific type of person. A big part of the assignment was to research what kinds of stereotypes we could face as the specific type of person. I choose to be me, a student athlete and I had in my speech I then had to break the barrier and shoe that the negative stereotypes about student athletes were false.

(15.034) These student writing samples are not very similar to any writing that I have done in the past. Although I have responded to scholarly topics I believe that I would have a bit of a hard time applying a rhetorical approach to the reading and responding to it with my own personal research. Previous experiences in the classroom such as extensive research projects, and argumentative essays have given me a familiar presence around writing samples, such as the ones provided. Conducting research on why women sports are less popularized has helped familiarize myself with voicing my opinion to reflect on the misconceptions of why toxic misogynic affects the viewing of these women sports. This task has helped me understand the idea of pursuing an interest in a topic, and understand why their are many taboos regarding a sport. 

(15.227) In the summer of 2016, I embarked on my golf journey. I transformed myself from a novice who consistently missed striking the golf ball at my local driving to succeeding in the gruesome tryout at the blustering Sandpiper golf club for the UCSB golf team Tryout. My journey to join the golf team provides me with the life context, and the reading in school provides me with the academic experience of "How to Read Like a Writer" and "understanding Genre in a rhetorical system." As a golfer, much like any other athlete, I would watch many highlights and documentaries of great players like Tiger Woods and Dustin Johnson, trying to figure out the underlying reason behind the type of shot and the choices they would make. Why do they usually hit the ball high in the United States while keeping it under the howling wind on the British links? The decision to go aggressive or defensive depends on the player's standing and the pressure to lead the tournament or chase the leader. One notable example is Tiger Woods' play on

(15.318) Firstly, the "Memory Project" would be the easiest assignment to me because in ninth grade English honours I had an assignment very similar to this one. In this assignment I wrote about my 18 and under boys water polo practice that I went to as a 13 year old girl who weighed no more than 110 pounds. I related this situation to my identity today because it taught me to not be intimidated by individuals who are more powerful than I am. Although I did not have to site a secondary source, and epigraph or provide a picture, I did have to site a primary source. My primary source was the coach of the 18 and under boys who initially wanted me to practice with his team. 

During my time in AP Lang. and Comp. and AP Literature, I have written an extensive amount of response essays that required an audience, prewriting, writing strategies and a thesis that connects to the whole essay. For example, during AP Lang. and Comp. one of our assignments were to write a response essay on "The Plastic Pink Flamingo". 

Rhetorical

(15.326) For IB English my senior year, we had to write about an object that really changed our lives and had a great impact on us, I chose a soccer ball. I chose this soccer ball because it really changed my life, sophomore year my grandpa died, during soccer conditioning. I got hurt that day, October 17, 2019, I remember it very well because all that happened and I wasn't able to play for soccer that year. I wrote on how I recovered from my injury and I started training again and I just connected with the soccer ball,  felt like a magician. I've never played soccer like that in my life so I was super excited and couldn't wait for the next season. I used the soccer ball as a gateway for happiness even after everything that happened to me, I still somehow found the light and it was shinning bright, ever since then I enjoyed soccer and it is something I adore and don't think of letting go. This is related to the "Memory Project"  because it is something that is now a part of my identity and is something so vivid to me

...

(16.421) Transitioning into UCSB is going to be an exciting and very different experience for me as a student. I went to a really small, charter high school where there were approximately 135 graduating seniors and I feel as though I want to move at a steady pace in my academics. I have always been an ambitious and hard working student that strived to take top classes at my school, however,  I want to make sure that I am comfortable my freshmen year. Thinking of doing club soccer, participating in clubs, all while maintaining grades come with have a manageable workload where I am understanding and learning new things at a comfortable pace as opposed to being totally confused like I feel like I would be in Writing 2. I feel as though I know a lot about the english language curriculum, however, I want to be comfortable in knowing that I know a basis of it but still learning new things. Having this idea, I feel as though I can accomplish this in Writing 1 as opposed to Writing 2. 

(16.440) Throughout high school, we were asked to read a multitude of books and articles and analyze their meaning through the form of writing. Bickmore's _Genre in the Wild _may seem a little bit more abstract and more unconventional of what was asked of in high school, however, the passages are fairly the same. Olivas' _Cupping the spark in our hands_ is more straight forward, as its clear direction is how students in their first year can learn how to write well-crafted research questions if they are taught how to do so, essentially the earlier the better. The past reading assignments that I have faced since then have prepared me to analyze passages such as these, as I am in my second year of college, my freshman classes furthered my knowledge of understanding passages and how to analyze them past the high school level. The courses that I have taken were very reading based, the majority of the class was spent reading the passage, analyzing information, and delivering it in a more concrete format.

(16.453) I would say that they are similar to what I have written in the past. In regards to disciplinary research analysis, I have taken IB Bussiness Management and IB Sports Exercise & Health Science which required an Internal Assessment (IA) to be written for each course. Each IA covered a concept learned in class that was further developed by a line of inquiry and was then supported by research and analysis. For IB Bussiness Management, I chose a company to research and posed a line of inquiry based on what my IA was focusing on. My line of inquiry was the following:

SHOULD MCDONALD’S RAISE THEIR HOURLY WAGE IN ORDER TO INCREASE THE COMPANY’S EMPLOYEE SATISFACTION AND PRODUCTIVITY?

This line of inquiry had to be supported by research, statistics, graphs, as well as various sources in order to determine whether it was true or false. With IB SEHS, I chose an experiment to conduct. My line of inquiry was the following:

IS THERE A CORRELATION BETWEEN AN INDIVIDUAL’S GENDER AND LEVEL OF ATHLETIC ABILITY IN REGARDS T

(16.664) Once I wrote an essay about Are S.T.E.M students just better than those non-S.T.E.M students. I was given some articles introducing the benefits of STEM and I was asked to write my own opinion on whether taking STEM prep classes makes a difference in high students' academic performance. I think this assignment is partly related to the one shown here because the topics of these two essays presented and the one I wrote about are all controversial and they all require personal experiences to support their arguments or statements. For these types of writing assignments, I have enough knowledge about the ongoing problems of society such as gender inequalities in certain areas to make my essay more informative.

(16.674) "GENDER STEREOTYPES AND LEARNING APPROACHES IN COMPUTER SCIENCE" IS A WRITING I AM VERY FAMILIAR WITH, AND I HAVE WRITTEN SOMETHING LIKE THIS BEFORE. MAO EXPLAINS HOW HER PASSION IS COMPUTER SCIENCE AND OVERALL HOW SHE CAN BEAT A STEREOTYPE THAT THE ENGINEERING FIELD WAS MADE FOR MEN. WITH THIS WRITING FORMAT, I HAVE WRITTEN A PERSONAL ESSAY SIMILAR TO THIS BEFORE. THE FORMAT INCLUDES AN INTRODUCTION, MANY BODY PARAGRAPHS AND A CONCLUSION, USING A GREAT AMOUNT OF PERSONAL EXPERIENCE AND EVIDENCE SHE HAS GATHERED TO GO AGAINST THE STEREOTYPE. SHE TALKED ABOUT HER APPROACH TO COMPUTER SCIENCE AND SHE EVEN INTERVIEWED HER FELLOW FEMALE COMPUTER SCIENCE PROFESSOR. I CAN RELATE THIS TO A WRITING EXPERIENCE I HAD MY JUNIOR YEAR, WHERE I CHOSE TO WRITE IN THE SCHOOLS NEWSPAPER ABOUT THE MISLEADING INFORMATION ABOUT CHEERLEADING.I EXPLAINED HOW CHEERLEADING IS IN FACT A SPORT AND HOW HARD EACH GIRL WORKS SUPER HARD IN ORDER TO BE ON THE SQUAD. IN THIS ARTICLE, I INTERVIEWED THE FOUR CAPTAINS FROM THE VARSITY SQUAD TH